# Deutsch-Jozsa Algorithm

The Deutsch-Jozsa Algorithm is another simple quantum algorithm that builds upon Deutsch's algorithm. It solves the following (again slightly contrived) problem: given a black box containing a function from $\{0, 1\}^n \rightarrow \{0, 1\}$, determine whether the mystery function is contant or balanced. That is determine if all inputs map to 0 or 1, or half map to 1 and half map to 0. In fact, Deutsch's algorithm is just this algorithm when $n = 1$. 

Classically, it would take $2^{n-1} + 1$ function calls to determine the answer. We can reduce the number of calls needed to only one--for all values of $n$!

We utilize the same strategy in Deutsch's algorithm to make a valid quantum oracle. We implement the oracle as $U_f|\textbf{x},y\rangle = |x, y\oplus \textbf{f(x)}\rangle$, where the bolded $x$ and $f(x)$ represent $n$ bits. get_mystery_function(n) takes $n$ as a parameter and generates a $2^{n+1} \times 2^{n+1}$ unitary matrix to be used as the oracle.

It works by first randomly deciding whether the function is contant or balanced. If it is balanced, half of the values from 0 to $2^n - 1$ are randomly chosen to be mapped to one. It then loops through the $2^{n+1}$ bitstrings in the domain. In this bitstring, the first bit represents the ancillary qubit $y$, while the following $n$ bits represent the remaining $n$ qubits. If the decimal representation of these remaining bits is in the randomly chosen list of inputs to map to one, then the ancillary qubit is XORd with 1, as per the definition of $U_f$. e.g. imagine that the function maps all inputs to 1. Then the $|0\textbf{x}\rangle$ state would map to $|0\otimes 1, \textbf{x}\rangle = |1\textbf{x}\rangle$. This mapping is different than is described in many textbooks because Qiskit has the 0th qubit as the least significant bit.

In [2]:
import itertools
import numpy as np
import random
from qiskit import *
from qiskit import Aer
from qiskit.quantum_info.operators import Operator

In [12]:
def get_mystery_function(n):
    get_bin = lambda x, n: format(x, 'b').zfill(n)

    op_list = []
    op = np.zeros((2**(n+1), 2**(n+1)))

    if (random.choice([True, False])):
        # constant
        if (random.choice([True, False])):
            print('Contant 1\'s')
            op_list = list(range(2**n))
        else:
            print('Constant 0\'s')
            op_list = []
    else:
        # balanced
        print('Balanced')
        op_list = random.sample(range(2**n), n)
   
    # check if last n bits are in op_list
    # if yes, replace first bit with first bit XOR 1
    for i in range(2**(n+1)):
        if (int(get_bin(i, n+1)[1:], 2) in op_list):
            op[i][int(str(int(get_bin(i, n+1)[0]) ^ 1) + get_bin(i, n+1)[1:], 2)] = 1
        else:
            op[i][int(str(int(get_bin(i, n+1)[0]) ^ 0) + get_bin(i, n+1)[1:], 2)] = 1
    return op

In [13]:
# number of bits to use in mystery function
n = 2

q = QuantumCircuit(n+1)
func = get_mystery_function(n)
op = Operator(func)

q.h(list(range(n)))
q.x(n)
q.h(n)
q.unitary(op, list(range(n+1)))
q.h(list(range(n)))

Constant 0's


In [14]:
q.draw()

In [15]:
meas = QuantumCircuit(n+1, n+1)
meas.barrier(range(n+1))
# map the quantum measurement to the classical bits
meas.measure(range(n+1),range(n+1))
qc = q+meas
qc.draw()

In [16]:
backend_sim = Aer.get_backend('qasm_simulator')
job_sim = execute(qc, backend_sim, shots=1024)
result_sim = job_sim.result()

In [17]:
# First  bits are the answer
counts = result_sim.get_counts(qc)
print(counts)

{'000': 512, '100': 512}


Let's start in the arbitrary state where the ancillary qubit is 1 and the other $n$ are irrelevant. 

$$|\Psi_0\rangle = |1\textbf{x}\rangle$$

Placing the ancillary qubit in a superposition gives.

$$|\Psi_1\rangle = H \otimes I^{\otimes n} = \Big[\frac{|0\rangle - |1\rangle}{\sqrt{2}}\Big]|\textbf{x}\rangle$$

After apply $U_f$ to $|\Psi_1\rangle$ we get:

$$|\Psi_2\rangle = U_f |\Psi_1\rangle = \Bigg( \frac{|0 \oplus f(\textbf{x})\rangle-|1 \oplus f(\textbf{x})\rangle}{\sqrt{2}} \Bigg)  |x\rangle = \Bigg( \frac{|f(\textbf{x})\rangle-|\overline{f(\textbf{x})}\rangle}{\sqrt{2}} \Bigg)  |\textbf{x}\rangle$$

$$= \begin{cases}
    \Big(\frac{|0\rangle - |1\rangle}{\sqrt{2}} \Big) |\textbf{x}\rangle,& \text{if $f(\textbf{x}) = 0$} \\
    \Big(\frac{|1\rangle - |0\rangle}{\sqrt{2}} \Big) |\textbf{x}\rangle,& \text{if $f(\textbf{x}) = 1$}
\end{cases}$$

$$= \Big(\frac{|0\rangle - |1\rangle}{\sqrt{2}} \Big) (-1)^{f(\textbf{x})}|\textbf{x}\rangle$$

This is almost exactly the same as Deutsch's algorithm, just that there is a register of $n$ bits to deal with now.

Lets do the matrix multiplication again because I like seeing the math work out. We'll work through the smallest non-trivial problem where $n = 2$. We'll use a constant operation so that we can see all of the top qubits interfere to collapse into $|\textbf{0}\rangle$. 

$$|\Psi_0\rangle = |1\rangle \otimes |0\rangle \otimes |0\rangle = \begin{bmatrix} 0 \\ 1 \end{bmatrix} \otimes \begin{bmatrix} 1 \\ 0 \end{bmatrix} \otimes \begin{bmatrix} 1 \\ 0 \end{bmatrix} = \begin{bmatrix} 0\\0\\0\\0\\1\\0\\0\\0 \end{bmatrix} \quad |100\rangle$$

$$|\Psi_1\rangle = H \otimes H^{\otimes n} |\Psi_0\rangle = \frac{1}{2\sqrt{2}} \begin{bmatrix} 1&1&1&1&1&1&1&1 \\ 1&-1&1&-1&1&-1&1&-1 \\ 1&1&-1&-1&1&1&-1&-1 \\ 1&-1&-1&1&1&-1&-1&1 \\ 1&1&1&1&-1&-1&-1&-1 \\ 1&-1&1&-1&-1&1&-1&1 \\ 1&1&-1&-1&-1&-1&1&1 \\ 1&-1&-1&1&-1&1&1&-1 \end{bmatrix} \begin{bmatrix} 0\\0\\0\\0\\1\\0\\0\\0 \end{bmatrix} = \frac{1}{2\sqrt{2}} \begin{bmatrix} 1\\1\\1\\1\\-1\\-1\\-1\\-1 \end{bmatrix} $$

$$|\Psi_2\rangle = U_f |\Psi_1\rangle = \begin{bmatrix} 0&0&0&0&1&0&0&0 \\ 0&0&0&0&0&1&0&0 \\ 0&0&0&0&0&0&1&0 \\ 0&0&0&0&0&0&0&1 \\ 1&0&0&0&0&0&0&0 \\ 0&1&0&0&0&0&0&0 \\ 0&0&1&0&0&0&0&0 \\ 0&0&0&1&0&0&0&0 \end{bmatrix} \frac{1}{2\sqrt{2}} \begin{bmatrix} 1\\1\\1\\1\\-1\\-1\\-1\\-1 \end{bmatrix} = \frac{1}{2\sqrt{2}} \begin{bmatrix} -1\\-1\\-1\\-1\\1\\1\\1\\1 \end{bmatrix}$$

$$|\Psi_3\rangle = I \otimes H^{\otimes n} |\Psi_2\rangle = \frac{1}{2} \begin{bmatrix} 1&1&1&1&0&0&0&0 \\ 1&-1&1&-1&0&0&0&0 \\ 1&1&-1&-1&0&0&0&0 \\ 1&-1&-1&1&0&0&0&0 \\ 0&0&0&0&1&1&1&1 \\ 0&0&0&0&1&-1&1&-1 \\ 0&0&0&0&1&1&-1&-1 \\ 0&0&0&0&1&-1&-1&1 \end{bmatrix} \frac{1}{2\sqrt{2}} \begin{bmatrix} -1\\-1\\-1\\-1\\1\\1\\1\\1 \end{bmatrix} = \frac{1}{\sqrt{2}}\begin{bmatrix} -1\\0\\0\\0\\1\\0\\0\\0 \end{bmatrix} \quad \Big[\frac{|1\rangle - |0\rangle}{\sqrt{2}}\Big] |00\rangle $$

The math works out again! Although it is kind of excessive.